In [1]:
import torch
import torch.nn.functional as F
from sklearn.model_selection import KFold
import numpy as np
import copy

import os
import sys
import pathlib
pth=str(pathlib.Path().absolute())

#Utilities
sys.path.append(pth)
from model_An import *
from util import *
from Data_loader_image import *

#Base de datos
#sys.path.append()
data=("/").join(pth.split("/")[:-2])+"/Data_Base/Metada_V6G_p1"
data_arg=("/").join(pth.split("/")[:-2])+"/Data_Base"


In [2]:
dataset=Rotated_Dataset(data_arg,"Metadata_V6G_p1")

In [3]:
train_set, val_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.7), len(dataset)-int(len(dataset)*0.7)])

In [7]:
def train_model(
    dataset,
        epochs,
        batch_size,
        use_cuda,
    folds=5,
        disable_tqdm=False,
        ):
    print("Reading dataset")
    #dset = MNIST(dset_folder,download=True)
    #DATASET
    #imgs = dset.data.unsqueeze(-1).numpy().astype(np.float64)
    #labels = dset.targets.numpy()
    
    #train_idx, valid_idx = map(np.array,util.split_dataset(labels))
    ind=np.arange(0,len(dataset))
    
    #INSIDE K-FOLD
    results={}
    for fold in range(folds):
        #generate train-test
        print("fold"+str(fold))
        
        i=fold

        ind=np.arange(0,len(dataset))
        indexes=ind[np.random.permutation(len(dataset))]
        L=len(dataset)
        test_idx=indexes[int(i*L/5):int((i+1)*L/5)]
        train_idx=np.delete(indexes,np.arange(int(i*L/5),int((i+1)*L/5)))
        
        model = GAT_ANE_(41,1)
        if use_cuda:
            model = model.cuda()
    
        opt = torch.optim.Adam(model.parameters())
    
        best_valid_acc = 0.
        best_model = copy.deepcopy(model)
    
        last_epoch_train_loss = 0.
        last_epoch_train_acc = 0.
        last_epoch_valid_acc = 0.
    
        interrupted = False
        
        train_dat=np.vectorize(lambda ind:dataset[ind],otypes=[object])(train_idx)
        train_graph=np.vectorize(lambda b:b["image_graph"])(train_dat)
        train_label=np.vectorize(lambda b:b["landmarks"])(train_dat)
        test_dat=np.vectorize(lambda ind:dataset[ind],otypes=[object])(test_idx)
        test_graph=np.vectorize(lambda b:b["image_graph"])(test_dat)
        test_label=np.vectorize(lambda b:b["landmarks"])(test_dat)
        
        loss_function = nn.MSELoss()
        for e in tqdm(range(epochs), total=epochs, desc="Epoch ", disable=disable_tqdm,):
            try:
                #train_losses, train_accs = train_(model, opt, train_graph, train_label,loss_function,
                                                  #batch_size=batch_size, use_cuda=use_cuda, disable_tqdm=disable_tqdm,)
                train_losses, train_accs =train_(model=model,
                                                 optimiser=opt,
                                                 graphs=train_graph,
                                                 labels=train_label,
                                                 use_cuda=use_cuda,
                                                 loss_function=nn.MSELoss(),
                                                 #batch_size=1,
                                                 batch_size=batch_size,
                                                 disable_tqdm=disable_tqdm,
                                                 profile=False)
            
                last_epoch_train_loss = np.mean(train_losses)
                last_epoch_train_acc = 100*np.mean(train_accs)
            except KeyboardInterrupt:
                print("Training interrupted!")
                interrupted = True
        
            #valid_accs = test_(model,test_graph,test_label,use_cuda,desc="Validation ", disable_tqdm=disable_tqdm,)
            valid_accs = test_(model=model,
                               graphs=test_graph,
                               labels=test_label,
                               use_cuda=use_cuda,
                               desc="Test ",
                               disable_tqdm=False)
                
            last_epoch_valid_acc = 100*np.mean(valid_accs)
        
            if last_epoch_valid_acc>best_valid_acc:
                best_valid_acc = last_epoch_valid_acc
                best_model = copy.deepcopy(model)
        
            tqdm.write("EPOCH SUMMARY {loss:.4f} {t_acc:.2f}% {v_acc:.2f}%".format(loss=last_epoch_train_loss, t_acc=last_epoch_train_acc, v_acc=last_epoch_valid_acc))
        
            if interrupted:
                break
    
        results[fold]={"train_acc":train_accs,"train_loss":train_losses,"valid_acc":valid_accs}
        save_model("best"+str(fold),best_model)
        save_model("last"+str(fold),model)
    return results
        


In [ ]:
results=train_model(
    dataset,
    epochs=5,
    batch_size=int(32),
    use_cuda=True,
    folds=5,
        disable_tqdm=False,
        )

Reading dataset
fold0


Epoch :   0%|          | 0/5 [00:00<?, ?it/s]
Instances :   0%|          | 0/15.125 [00:00<?, ?it/s]
                                                      
Epoch :   0%|          | 0/5 [00:00<?, ?it/s]?, ?it/s]
Instances :   7%|▋         | 1/15.125 [00:00<00:03,  4.64it/s]

139.4596	13945.96%	[-0.16217956] (x[448])



                                                              
Epoch :   0%|          | 0/5 [00:00<?, ?it/s]00:03,  4.64it/s]
Instances :  13%|█▎        | 2/15.125 [00:00<00:02,  4.60it/s]

138.6987	13869.87%	[-0.12914923] (x[421])


...

131.4730	13147.30%	[0.13175936] (x[480])



                                                              
Epoch :  60%|██████    | 3/5 [00:19<00:12,  6.45s/it]4.32it/s]
Instances :  13%|█▎        | 2/15.125 [00:00<00:03,  4.25it/s]

131.2991	13129.91%	[0.13513623] (x[480])



                                                              
Epoch :  60%|██████    | 3/5 [00:20<00:12,  6.45s/it]4.25it/s]
Instances :  20%|█▉        | 3/15.125 [00:00<00:02,  4.33it/s]

131.1659	13116.59%	[0.13852555] (x[480])


In [8]:
model = GAT_ANE(5,1)
model

GAT_ANE(
  (GAT_layers): ModuleList(
    (0): GATLayerMultiHead(
      (GAT_heads): ModuleList(
        (0): GATLayerEdgeSoftmax(
          (f): Linear(in_features=10, out_features=32, bias=True)
          (w): Linear(in_features=10, out_features=1, bias=True)
        )
        (1): GATLayerEdgeSoftmax(
          (f): Linear(in_features=10, out_features=32, bias=True)
          (w): Linear(in_features=10, out_features=1, bias=True)
        )
      )
    )
    (1): GATLayerMultiHead(
      (GAT_heads): ModuleList(
        (0): GATLayerEdgeSoftmax(
          (f): Linear(in_features=128, out_features=64, bias=True)
          (w): Linear(in_features=128, out_features=1, bias=True)
        )
        (1): GATLayerEdgeSoftmax(
          (f): Linear(in_features=128, out_features=64, bias=True)
          (w): Linear(in_features=128, out_features=1, bias=True)
        )
      )
    )
    (2): GATLayerMultiHead(
      (GAT_heads): ModuleList(
        (0): GATLayerEdgeSoftmax(
          (f): Linea

In [10]:
model = GAT_ANE_(5,1)
model

GAT_ANE_(
  (GAT_layers): ModuleList(
    (0): GATLayerEdgeSoftmax(
      (f): Linear(in_features=10, out_features=32, bias=True)
      (w): Linear(in_features=10, out_features=1, bias=True)
    )
    (1): GATLayerEdgeSoftmax(
      (f): Linear(in_features=64, out_features=64, bias=True)
      (w): Linear(in_features=64, out_features=1, bias=True)
    )
    (2): GATLayerEdgeSoftmax(
      (f): Linear(in_features=128, out_features=64, bias=True)
      (w): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (MLP_layers): ModuleList(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=1, bias=True)
  )
)

array([[1, 1, 1, 1]])